In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading PL data
PL_dict = zload(data_folder + 'PL_dict.pkz') 

In [5]:
# PL_dict

In [6]:
mu_1 = PL_dict['PL_AM']  # normal PL for AM period
mu_2 = PL_dict['PL_MD']  # normal PL for MD period
mu_3 = PL_dict['PL_PM']  # normal PL for PM period
mu_4 = PL_dict['PL_NT']  # normal PL for NT period

mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

mu_2 = mu_adjust(mu_2)  # normal PL
mu_02, mu2, mu_12, P2, G_12, H_12, U_12 = ChainGen_(mu_2)

mu_3 = mu_adjust(mu_3)  # normal PL
mu_03, mu3, mu_13, P3, G_13, H_13, U_13 = ChainGen_(mu_3)

mu_4 = mu_adjust(mu_4)  # normal PL
mu_04, mu4, mu_14, P4, G_14, H_14, U_14 = ChainGen_(mu_4)

<string>:486: RuntimeWarning: covariance is not positive-semidefinite.


In [7]:
beta = 0.001
G_list = [G_11, G_12, G_13, G_14]
H_list = [H_11, H_12, H_13, H_14]
U_list = [U_11, U_12, U_13, U_14]

eta_wc = {}
eta_Sanov = {}

# Get thresholds for Hoeffding's test corresponding to sample length n  
def etaWC(n):
    return HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)

In [8]:
# Loading test data

with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [9]:
sample_path_jam_test_data_sorted_trans['0']

{u'alphaTrans': 7.0,
 u'startTime': 300.0742,
 u'uuid': [u'4a27354e-e8c6-373c-9863-f6bfe6d7a4d4',
  u'd836e23a-c958-37a7-b126-c70258b61a7a']}

In [10]:
i = 0  # index of windows
sam_gap = 100
win_size = 600
win_list = []
while i * sam_gap + win_size < 24 * 3600:
    win_list.append((i * sam_gap, i * sam_gap + win_size))
    i += 1

In [11]:
i

858

In [12]:
win_num = len(win_list)
jam_num = len(sample_path_jam_test_data_sorted_trans)

win_jam = {}
for i in range(win_num):
    key = str(i)
    value = []
    for j in range(jam_num):
        if ((sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] >= win_list[i][0]) \
            and (sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] < win_list[i][1])):
            value.append(str(j))
    win_jam[key] = value

In [13]:
win_jam['857'], win_list[857][0], win_num 

(['3733',
  '3734',
  '3735',
  '3736',
  '3737',
  '3738',
  '3739',
  '3740',
  '3741',
  '3742',
  '3743',
  '3744',
  '3745',
  '3746',
  '3747',
  '3748',
  '3749',
  '3750',
  '3751',
  '3752',
  '3753',
  '3754'],
 85700,
 858)

In [14]:
sample_path_jam_test_data_sorted_trans[win_jam['100'][0]]['startTime']

10500.0035

In [15]:
KL_list = []  # The list of the KL divergences; each is for a detection window
etaWC_list = []
for i in range(win_num):
    sample_path_jam = [sample_path_jam_test_data_sorted_trans[key]['alphaTrans'] for key in win_jam[str(i)]]
    n = len(sample_path_jam)
    if (n != 0):
        etaWC_list.append(etaWC(n))
        KL_1 = KL_est(sample_path_jam, mu_1)
        KL_2 = KL_est(sample_path_jam, mu_2)
        KL_3 = KL_est(sample_path_jam, mu_3)
        KL_4 = KL_est(sample_path_jam, mu_4)
        KL_list.append(min([KL_1, KL_2, KL_3, KL_4]))
    else:
        etaWC_list.append(1e-6)
        KL_list.append(0)

In [16]:
win_jam[str(100)], win_list[100][1]

(['207', '208', '209', '210', '211', '212', '213', '214'], 10600)

In [25]:
time_range = [win_list[i][0] / 60.0 for i in range(win_num)]
plot_points(time_range, KL_list, etaWC_list)
plt.ylabel('divergence')
plt.xlabel('time (min)')
pylab.ylim(-0.1, max(etaWC_list)+0.5)
pylab.xlim(0, max(time_range) + 10)
plt.savefig(data_folder + 'detec_results_WC_robust.pdf')
plt.show()

In [18]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(858, 858, 858, [0.0, 1.6666666666666667])

In [19]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [20]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)

In [21]:
sample_path_jam_test_data_sorted['1181']

{u'alpha': 2.0,
 u'length': 2216.0,
 u'numPts': 9.0,
 u'speed': 7.383333333333333,
 u'startTime': 49500.1235,
 u'uuid': u'f477324a-b20e-3ec8-8407-9ebb8ae94154'}

In [22]:
len(set(potential_ano_list)), set(potential_ano_list)

(61,
 {'1167',
  '1168',
  '1169',
  '1170',
  '1171',
  '1172',
  '1173',
  '1174',
  '1175',
  '1176',
  '1177',
  '1178',
  '1179',
  '1180',
  '1181',
  '1182',
  '1183',
  '1184',
  '1185',
  '1186',
  '1187',
  '1188',
  '1189',
  '1190',
  '1191',
  '1192',
  '1193',
  '1194',
  '1195',
  '1196',
  '1197',
  '1198',
  '1199',
  '1200',
  '1201',
  '1202',
  '1203',
  '1204',
  '1205',
  '1206',
  '1207',
  '1208',
  '1209',
  '1210',
  '1211',
  '1212',
  '1213',
  '1214',
  '1215',
  '1216',
  '1217',
  '1218',
  '1219',
  '1220',
  '1221',
  '1222',
  '1223',
  '1224',
  '1225',
  '1226',
  '1227'})

In [23]:
len(sample_path_jam_test_data_sorted_trans)

3755

In [24]:
50000 / 3600.0, 60000 / 3600.0

(13.88888888888889, 16.666666666666668)